# Source : VILLE INTELLIGENTE

## Chargement des librairies pour le scraping

In [ ]:
import bs4 # librairie de scraping
from bs4 import BeautifulSoup # librairie de scraping
import requests # librairie de scraping
import urllib.request # librairie de scraping
import pandas as pd # librairie pour manipuler des dataframes
import numpy as np # librairie de datascience

from tqdm import tqdm, tqdm_notebook # permet de connaître le temps d'exécution de nos cellules
import re # librairie d'opérations sur les opérations rationnelles

## Récupèrer le code des pages

In [ ]:
def get_pages_sc(base,key_word: str = 'None',nb_pages: int = 1):
  """# Documentation:
  # cette fonction récupère le code source d'une URL
    
  # Parameters:
  # base: url du site où se trouvent les articles à récupérer
  # key_word
  # nb_pages
        
  # Out:
    # pages_list: liste du contenu des pages
  """

  # dictionnaire d'User-Agent, que l'on spécifiera à notre requêteur
  header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

  # à changer en fonction de la source
  base = base + '/' + key_word

  pages_list = []
  cpt: int = 0

  for i in tqdm_notebook(range(nb_pages)):
    # Simulate a webdriver instance and get the source page
    if nb_pages > 1:
      # à changer en fonction de la source
      req = requests.get(base +  '/?start_liste=' + str(cpt),headers = header)
      # transform it with the BS' html parser
      soup = bs4.BeautifulSoup(req.text,'html.parser')
      pages_list.append(soup)
      cpt += 20
    else:
      req = requests.get(base,headers = header)
      # transform it with the BS' html parser
      soup = bs4.BeautifulSoup(req.text,'html.parser')
      pages_list.append(soup)

  return pages_list

# programme principal de la fonction get_pages_sc
pages_list = get_pages_sc('https://www.villeintelligente-mag.fr/search/','data',6)
pages_list

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



6


[<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
 
 <html lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml">
 <head>
 <title>Data | Recherche | Ville Intelligente Mag</title>
 <meta content="text/html; Charset=UTF-8" http-equiv="Content-Type"/>
 <meta content="Smart City, ville intelligente, objets connectés, iOT, technologie, urbain, environnement, Angers, Health Care, santé, économie numérique, silver économie, développement, applications" name="keywords"/>
 <meta content="Présentiel et distanciel : comment numériser, gérer, automatiser et distribuer efficacement les informations ? Avec Ouest Data Hub, la médecine de demain est déjà une réalité Le 1er janvier, L’IG..." name="description"/>
 <meta content="47.476096;-0.5492282" name="geo.position"/>
 <link href="/var/style/style_7.css?v=1608283813" rel="stylesheet" type="text/css"/>
 <link href="/var/style/style.26888365.css?v=1550515270" rel="stylesheet" type="t

## Récupérer les liens des articles

In [27]:
# pas de class dans href
def get_art_url(pages_list,tag,attr = None):
  """# Documentation:
  # cette fonction récupère les liens des arcticles
   
  # Parameters:
    # pages_list
    # tag: balise de l'élément à récupérer
    # attr: attribut
        
  # Out:
    # url_list: liste des URL menant à chaque articles
  """
    
  # initialisation de la liste contenant les liens vers les articles
  url_list = []

  for page in tqdm_notebook(pages_list):  
    # on cherche les balises a, de valeur class block-actus-content-over block-actus-more
    hyperlinks_list = page.find_all(tag,attr)
    for hyperlink in hyperlinks_list:
      # on ajoute à la liste des url l'attribut href, qui est l'url de l'article
      url_list.append(hyperlink)
  # le set permet de supprimer les potentiels doublons
  url_list = list(set(url_list))
    
  for i in range(0,len(url_list)-1):
    if url_list[i] == '#':
      del url_list[i]            
        
  return url_list

# programme principal de la fonction get_art_url
# liens des articles
url_list = get_art_url(pages_list,'h3',{'class':'titre'})
url_list

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[<h3 class="titre" style="margin-bottom:5px">
 <a href="/Coronavirus-Et-si-Internet-tombait-en-panne_a864.html">#Coronavirus : Et si Internet tombait en panne ?</a>
 </h3>, <h3 class="titre" style="margin-bottom:5px">
 <a href="/Pendant-le-WEF-et-la-Connected-Week-d-Angers-on-parlera-aussi-Smart-City_a318.html">Pendant  le WEF et la Connected Week d’Angers, on parlera aussi Smart City</a>
 </h3>, <h3 class="titre" style="margin-bottom:5px">
 <a href="/Copenhague-cite-europeenne-exemple-de-ville-intelligente-et-durable_a830.html">Copenhague, cité européenne, exemple de ville intelligente et durable</a>
 </h3>, <h3 class="titre" style="margin-bottom:5px">
 <a href="/Filiere-numerique -les-metropoles-du-Grand-Ouest-veulent-debaucher-les-talents-parisiens_a505.html">Filière numérique : les métropoles du Grand Ouest veulent débaucher les talents parisiens</a>
 </h3>, <h3 class="titre" style="margin-bottom:5px">
 <a href="/Droit-Le-tracage-numerique-de-la-population-en-temps-de-coronavirus-p

In [32]:
def get_href_art_url(url_list):
  """# Documentation:
  # cette fonction filtre les liens
   
  # Parameters:
    # url_list
        
  # Out:
    # url_list: liste des URL menant à chaque articles
  """
  href_list = []

  for article in url_list:  
    # on cherche les balises a, de valeur class block-actus-content-over block-actus-more
    hyperlinks_list = article.find_all('a')
    for hyperlink in hyperlinks_list:
      # on ajoute à la liste des URL l'attribut href, qui est l'url de l'article
      href_list.append(hyperlink['href'])
  # le set permet de supprimer les potentiels doublons
  href_list = list(set(href_list))

  return href_list

# programme principal de la fonction get_href_art_url
# liens des articles
href_list = get_href_art_url(url_list)
href_list

['/ADN-Ouest-lance-l-Observatoire-Regional-des-Competences-du-Numerique_a125.html',
 '/Mobilite\xa0-Le-court-voiturage-de-Karos-plebiscite-par-les-Franciliens_a415.html',
 '/Les-Telecoms-sont-ils-les-moteurs-de-la-transformation-numerique-de-nos-villes_a637.html',
 '/Le-vehicule-connecte-antichambre-du-vehicule-autonome_a729.html',
 '/SUEZ-place-l-innovation-au-coeur-de-la-protection-des-ressources_a172.html',
 '/Transformation-numerique-comment-l-economie-de-la-donnee-impacte-les-architectures-informatiques_a376.html',
 '/La-France-taxera-les-GAFA-des-le-1er-janvier-2019_a619.html',
 '/Copenhague-cite-europeenne-exemple-de-ville-intelligente-et-durable_a830.html',
 '/YUKA-l-appli-sympa-qui-vous-aide-a-manger-plus-sain_a355.html',
 '/Amsterdam-Du-WiFi-dans-des-nichoirs-intelligents-quand-l-air-est-sain_a86.html',
 '/Developper-une-politique-locale-de-la-donnee_a246.html',
 '/En-France-des-projets-de-Smart-City-dans-une-vingtaine-de-villes_a65.html',
 '/Avec-Num-Alim-l-Empire-contre-att

## Récupérer le code des articles

In [ ]:
def get_art_sc(url_list):
  """# Documentation:
  # cette fonction récupère le code source des articles
    
  # Parameters:
    # url_list: url des articles
        
  # Out:
    # art_list: liste du contenu des articles
  """
    
  # initialisation de la liste contenant les liens vers les articles
  art_list = []
    
  # boucle qui parcourt le code source des articles
  for url in tqdm_notebook(url_list):
    # on récupère la page web
    req = requests.get(url)
    # on récupère le code source
    soup = bs4.BeautifulSoup(req.text,'html.parser')
    art_list.append(soup)
    
  return art_list

# programme principal de la fonction get_art_sc
art_list = get_art_sc(url_list)
art_list

## Récupérer le titre de l'article

In [ ]:
def get_art_title(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère le titre de l'article
    
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr: attribut
        
  # Out:
    # title_list[0]: premier titre de l'article
  """
    
  # renvoie une liste des balises trouvées, avec les titres
  title_source = art_sc.find_all(tag,attr)
  title_list = []
   
  # on enlève les balises, on garde uniquement le contenu textuel, soit le titre
  for i in title_source:
    title_list.append(i.text)
    
  # si on a trouvé au moins un titre, alors on renvoie le 1er
  if title_list :
    return title_list[0]
  # sinon on renvoie rien
  else :
    return ''

## Récupérer la date de publication

In [ ]:
def get_art_extract_datetime(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère la date de publication
    
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr: attribut
        
  # Out:
    # list_date[0]: première date de l'article
  """
    
  date_source = art_sc.find_all(tag,attr)
  list_date = []
    
  for i in date_source:
    list_date.append(i.text)
    
  if list_date:
    return list_date[0]
  else:
    return ''

## Récupérer les tags de l'article

In [ ]:
def get_art_tag(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère les tags de l'article
    
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr: attribut
        
  # Out:
    # list_tag : liste des tags de l'article
  """
        
  tag_source = art_sc.find_all(tag,attr)  
  list_tag = []
    
  for tag in tag_source:
    list_tag.append(tag.text)
            
  if list_tag:
    return list_tag[0]
  else:
    return ''

  return list_tag

## Récupérer l'auteur de l'article

In [ ]:
def get_art_auth(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère l'auteur de l'article
    
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr
       
  # Out:
    # list_aut: liste des auteurs
  """

  aut_source = art_sc.find_all(tag,attr)
  list_aut = []

  for auteur in aut_source:
    if auteur:
      list_aut.append(auteur.text)

  return list_aut


## Récupérer le contenu textuel

In [ ]:
def get_art_content(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère le contenu textuel de l'article
   
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr
        
  # Out:
    # content
  """

  content_source = art_sc.find_all(tag,attr)
  content = []
    
  for i in content_source:
    content.append(i.text)
    
  if content :
    return content[0]
  else :
    return ''

## Récupérer le contenu (avec balise)

In [ ]:
def get_art_content_html(art_sc,tag,attr = None):
  """# Documentation:
  # cette fonction récupère le contenu (avec balise) de l'article
   
  # Parameters:
    # art_sc: code source d'un article
    # tag
    # attr
        
  # Out:
    # content_html
  """

  content_source_html = art_sc.find_all(tag,attr)
  content_html = []
    
  for i in content_source_html:
    content_html.append(str(i))
    
  if content_html :
    return content_html[0]
  else :
    return ''

## Test des fonctions sur la source ADF

In [ ]:
# création du tableau pour insérer les données à scraper
df_adf = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag'])
df_adf

i = 0

# code source de la page
code_sc = get_pages_sc('http://www.departements.fr/actualites/?filter-theme=119&filter-departements=&filter-categorie=&search=')

# liste des url des articles
url_list = get_art_url(code_sc,'a',{'class':'block-actus-content-over block-actus-more'})

# liste de la source des articles
art_list = get_art_sc(url_list)

# on boucle sur chacun des articles et on récupère les informations souhaitées
for soup in tqdm_notebook(art_list):

    art_content = get_art_content(soup,'p')
    
    art_content_html = get_art_content_html(soup,'p')

    art_extract_datetime = get_art_extract_datetime(soup,'span',{'class':'block-actus-date'})
    
    art_lang = 'fr'

    art_title = get_art_title(soup,'h1',{'class':'col-md-8 col-lg-9'})
    
    art_url = url_list[i]
    
    src_name = 'departements'
    
    src_type = 'xpath_source'
    
    src_url = 'www.departements.fr'
    
    src_img = ''

    art_auth = ''
    
    art_tag = get_art_tag(soup,'span',{'class':'block-actus-tag'})
    
    # alimentation du dataframe
    df_adf.loc[len(df_adf)] = [art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag]
    i += 1
    
# affichage du tableau
df_adf

# extraction du df au format csv
#df_adf.to_csv('gdrive/MyDrive/scraping_adf.csv', index = False, sep = ';')
    
# au format json
# df_adf.to_json('gdrive/MyDrive/scraping_adf.json')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,Inscrite dans le plan sanitaire global de déco...,"<p><strong><p style=""text-align: justify;""><st...",17.06.2020,fr,,http://www.departements.fr/stopcovid-demarche-...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
1,"Déclarer ses revenus, créer son entreprise, fa...","<p><strong><p style=""text-align: justify;""><st...",08.10.2020,fr,,http://www.departements.fr/illectronisme-depar...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
2,Réuni sous la présidence de Jacqueline Gouraul...,"<p><strong><p style=""text-align: justify;""><st...",05.11.2020,fr,,http://www.departements.fr/prix-territoria-202...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
3,"Faire les courses pour ses voisins âgés, coudr...","<p><strong><p style=""text-align: justify;"">Fai...",17.06.2020,fr,,http://www.departements.fr/plateformes-de-soli...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
4,En parallèle des Etats généraux du numérique p...,"<p><strong><p style=""text-align: justify;""><st...",08.10.2020,fr,,http://www.departements.fr/2020-rentree-classe...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
5,"Le 13 novembre dernier, plus de 400 élus locau...","<p><strong><p style=""text-align: justify;""><st...",27.11.2019,fr,,http://www.departements.fr/prix-territoria-201...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
6,"Le 28 novembre dernier, le Département de la S...","<p><strong><p style=""text-align: justify;""><st...",11.12.2019,fr,,http://www.departements.fr/cinq-departements-s...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
7,Le Conseil départemental des jeunes de la Côte...,"<p><strong><p style=""text-align: justify;""><st...",21.10.2020,fr,,http://www.departements.fr/jeunes-cote-doriens...,departements,xpath_source,www.departements.fr,,,Innovation et Numérique
